In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords, twitter_samples
import re
import string
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [14]:
positive_dataset=twitter_samples.strings('positive_tweets.json')
negative_dataset=twitter_samples.strings('negative_tweets.json')
dataset=positive_dataset+negative_dataset

In [8]:
stop_words=stopwords.words('english')
stemmer=PorterStemmer()
tokenizer=TweetTokenizer(preserve_case=False,
                         strip_handles=True,
                         reduce_len=True)
def preprocess_text(text):
    text=re.sub(r'^RT[\s]+','',text)
    text=re.sub(r'https?:\/\/.*[\r\n]*','',text)
    text=re.sub(r'#','',text)
    word_tokens=tokenizer.tokenize(text)
    filtered_sentence=[stemmer.stem(w) for w in word_tokens if not w in stop_words and not w in string.punctuation]
    return filtered_sentence

prep_corpus=[preprocess_text(text) for text in dataset]

In [47]:
positive_corpus=[word for words in prep_corpus[:5000] for word in words]
negative_corpus=[word for words in prep_corpus[5000:] for word in words]
unique_words=list(set([w for words in prep_corpus for w in words]))
def extract_freq(item,unique_words=unique_words):
    item_freq={word:0 for word in unique_words}
    for w in item:
        if w in unique_words:
            item_freq[w]+=1
    return item_freq

positive_freq=extract_freq(positive_corpus)
negative_freq=extract_freq(negative_corpus)

67496

In [49]:
def extract_features(values):
  unique_vals=list(set(values))
  result_positive=sum([positive_freq[word] for word in unique_vals])
  result_negative=sum([negative_freq[word] for word in unique_vals])

  return [1,result_positive,result_negative]

results=np.array([extract_features(item) for item in prep_corpus])


In [50]:
zeros=np.zeros(len(results[:5000]))
ones=np.ones(len(results[5000:]))
a=np.append(np.ones((len(results[:5000]),1)),np.zeros((len(results[5000:]),1)),axis=0)
results

array([[   1, 3764,   72],
       [   1, 4464,  517],
       [   1, 3759,  160],
       ...,
       [   1,   34, 4640],
       [   1,   10, 4585],
       [   1,  116,  582]])

In [51]:
prep_array=np.hstack([results,a]).astype(int)
data=pd.DataFrame(prep_array,columns=['bias','positive','negative','sentiment'])

In [52]:
data.to_csv('sentiment_dataset.csv')